In [1]:
# # for colab

# from google.colab import drive
# import os

# drive.mount('/content/drive')
# main_dir = '/content/drive/MyDrive/Graduate Project'
# os.chdir(main_dir)
# os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/Graduate Project'

In [1]:
import pandas as pd
import geopandas as gpd

zillow_df = gpd.read_file('../dataset/raw/2. zillow_cleaned.geojson')
zillow_df

,zpid,city,submarket,zipcode,address,price,like,view,duration,day,single,parking,bathroom,bedroom,age,living,description,geometry
0,4171164,CH,South,60617,10905 S Avenue H,309000,0,20,134,64,0,2.5,4.0,4.0,81.0,2376.0,Tastefully and professionally updated 4BR / 3....,POINT (-87.53252 41.69706)
1,80851025,CH,South,60616,475 W 24th St APT 5H,265000,0,44,50,7,1,1.0,2.0,2.0,19.0,900.0,Experience City living as its finest in this t...,POINT (-87.63969 41.84879)
2,2127908883,CH,West,60612,2827 W Congress Pkwy APT 3,230000,0,7,90,83,1,1.0,2.0,2.0,17.0,1300.0,"Offering exceptional comfort, convenience, and...",POINT (-87.69721 41.87457)
3,60265291,CH,North,60614,2712 N Ashland Ave #4A,460271,0,20,25,64,1,1.0,2.0,2.0,24.0,1700.0,Looking for an updated extra-wide 2 bed/2 bath...,POINT (-87.66872 41.93097)
4,3700052,CH,North,60640,5415 N Sheridan Rd APT 306,319000,2,112,37,40,1,1.0,2.0,2.0,51.0,1250.0,"Discover the allure of this 2-bed, 2-bath gem ...",POINT (-87.65439 41.97993)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10106,20383564,LA,West LA,90045,8117 Chase Ave,1811700,4,186,41,7,0,4.0,3.0,4.0,78.0,1892.0,Welcome to this impressive Westchester home vi...,POINT (-118.41125 33.96461)
10107,20510134,LA,West LA,90067,10128 Empyrean Way APT 302,2100000,0,17,178,79,1,2.0,4.0,2.0,45.0,2498.0,"From the foyer, enter to a sophisticated livin...",POINT (-118.40809 34.05322)
10108,19905411,LA,San Fernando Valley,91335,7428 Vanalden Ave,815000,0,59,5,47,0,2.0,1.0,3.0,72.0,1670.0,Major Equity opportunity in the City of Reseda...,POINT (-118.54884 34.20552)
10109,302794497,LA,West LA,90094,12678 Millennium,3715000,1,140,78,12,0,3.0,5.0,4.0,7.0,3639.0,Embrace luxury living and the best in Playa Vi...,POINT (-118.41388 33.97678)


# Llama

In [ ]:
# from google.colab import userdata
from huggingface_hub import login

# hf_token = userdata.get('HF_TOKEN')
# login(token=hf_token)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch
from transformers import pipeline

device = "cuda" if torch.cuda.is_available() else "cpu"
llama_32_1b = "meta-llama/Llama-3.2-1B-Instruct"
llama_32_3b = "meta-llama/Llama-3.2-3B-Instruct"
generator = pipeline(model=llama_32_3b, device=device, torch_dtype=torch.bfloat16)

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

# Prompt Design

In [ ]:
# Rating Scale
rating_scale = {
    "1": "Implicit mention or general statement without specific details.",
    "2": "Explicit mention or detailed emphasis with specific examples."
}

# Contexts to extract
contexts = {
    "Physical Housing Quality": "Describes the structural condition and quality of the house itself, including materials and build quality.",
    "Neighborhood Characteristics": "Describes the safety, community features, and overall vibe of the neighborhood, important for families and those valuing community involvement.",
    "Accessibility": "Provides information about the property's geographical location and its accessibility to nearby amenities and transportation options.",
    "New Investment-Remodeled or Updated": "Highlights any recent renovations or updates that have been made to the property, indicating its current market readiness.",
    "Occupancy Status": "Describes the current occupancy status of the property or move-in availability, helping buyers know if immediate occupancy is possible.",
    "Natural Environment": "Describes the property's proximity to natural scenery or parks, appealing to those seeking a nature-friendly living environment.",
    "Size": "Describes the size of the home and how space is utilized, focusing on suitability for family size or lifestyle needs.",
    "Signaling": "Refers to any marketing techniques or cues used by realtors to highlight certain aspects of the property.",
    "Opinion": "Includes subjective opinions or impressions provided by realtors about the property, which may influence potential buyers."
}

# Mention in Description
examples = {
    "Physical Housing Quality": {
        1: [
            "The house seems to be in decent shape, though it may require some touch-ups.",
            "Overall, the property is in fair condition with potential for improvements.",
            "The structure appears solid, but some areas might need attention."
        ],
        2: [
            "The property boasts a brand-new roof and recently installed energy-efficient windows, ensuring top-notch physical housing quality.",
            "Features high-quality hardwood flooring and freshly painted walls throughout.",
            "Equipped with modern appliances and a recently renovated kitchen."
        ]
    },
    "Neighborhood Characteristics": {
        1: [
            "The neighborhood is generally nice and family-friendly.",
            "Located in a quiet area with a good sense of community.",
            "Surroundings are pleasant with a few local amenities nearby."
        ],
        2: [
            "Located in the heart of the vibrant Greenfield community, known for its excellent schools and active neighborhood association.",
            "Situated in a highly sought-after area with top-rated schools and parks.",
            "This friendly neighborhood offers a mix of cultural attractions and community events."
        ]
    },
    "Accessibility": {
        1: [
            "The property is somewhat accessible to local amenities.",
            "Public transport options are available within a reasonable distance.",
            "Basic services and shops are not too far away."
        ],
        2: [
            "Just a 5-minute walk from the Central Park subway station and close to major shopping centers.",
            "Conveniently located near major highways and public transportation hubs.",
            "Within walking distance to downtown amenities and public transit options."
        ]
    },
    "New Investment-Remodeled or Updated": {
        1: [
            "The house has had some updates over the years.",
            "Minor renovations have been made recently.",
            "Some parts of the property have been refreshed."
        ],
        2: [
            "Recently remodeled with a modern open-concept kitchen and updated bathrooms featuring marble countertops.",
            "Fully renovated with new electrical wiring and plumbing systems.",
            "Upgraded with state-of-the-art smart home technology throughout."
        ]
    },
    "Occupancy Status": {
        1: [
            "The home is available soon.",
            "Expected to be ready for occupancy shortly.",
            "Potential for quick move-in after purchase."
        ],
        2: [
            "Currently vacant and available for immediate move-in, perfect for buyers needing quick occupancy.",
            "Ready for immediate occupancy with no prior arrangements needed.",
            "Move-in ready condition, available as soon as paperwork is completed."
        ]
    },
    "Natural Environment": {
        1: [
            "There are some parks nearby.",
            "Green spaces can be found within driving distance.",
            "A few natural spots are accessible from the property."
        ],
        2: [
            "Nestled near the scenic Riverwalk Park, offering beautiful trails and picnic areas just steps away.",
            "Surrounded by lush greenery with direct access to hiking trails and nature reserves.",
            "Close proximity to the serene Lakeview Park, ideal for outdoor enthusiasts."
        ]
    },
    "Size": {
        1: [
            "The home is spacious enough for a small family.",
            "Adequate space for comfortable living arrangements.",
            "Offers sufficient room for basic needs."
        ],
        2: [
            "This 3,000 square foot home includes four bedrooms and a finished basement, ideal for larger families.",
            "Generous living space with expansive rooms and ample storage options.",
            "Spacious layout featuring multiple living areas and an oversized garage."
        ]
    },
    "Signaling": {
        1: [
            "Charming home in a desirable area.",
            "A must-see property!",
            "Perfect starter home!"
        ],
        2: [
            "This stunning property won't last long on the market! Unique architectural style and prime location.",
            "An exceptional find with unparalleled features!",
            "Luxury living at its finest in a prestigious neighborhood."
        ]
    },
    "Opinion": {
        1: [
            "Cozy and inviting atmosphere.",
            "A great place to call home.",
            "Feels like home from the moment you step inside."
        ],
        2: [
            "This property exudes warmth and character, offering an inviting atmosphere.",
            "A harmonious blend of comfort and elegance throughout.",
            "Every corner of this home speaks of quality craftsmanship."
        ]
    }
}

# Reason for Rating
justifications = {
    "Physical Housing Quality": {
        1: [
            "The mention of the house being in decent shape but needing touch-ups suggests a general statement about the structure without specific details, fitting a rating of 1.",
            "Describing the property as having potential for improvements implies a broad assessment of quality, which aligns with an implicit mention.",
            "Noting that some areas might need attention provides a vague reference to the house's condition, justifying a rating of 1."
        ],
        2: [
            "The specific mention of a brand-new roof and energy-efficient windows provides clear examples of high-quality features, warranting a rating of 2.",
            "Highlighting high-quality hardwood flooring and freshly painted walls offers explicit details about the property's condition.",
            "Mentioning modern appliances and a renovated kitchen gives unmistakable references to quality improvements, thus rated as 2."
        ]
    },
    "Neighborhood Characteristics": {
        1: [
            "Describing the neighborhood as generally nice and family-friendly is a broad statement without specifics, suitable for a rating of 1.",
            "Mentioning a quiet area with a good sense of community provides an implicit understanding of neighborhood characteristics.",
            "The reference to pleasant surroundings with local amenities nearby is vague and requires interpretation, aligning with rating 1."
        ],
        2: [
            "Specifying the Greenfield community with its schools and association gives detailed emphasis on neighborhood features.",
            "Identifying top-rated schools and parks in the area provides clear examples, fitting a rating of 2.",
            "Detailing cultural attractions and community events offers explicit references to neighborhood characteristics."
        ]
    },
    "Accessibility": {
        1: [
            "Stating that the property is somewhat accessible to amenities is a general comment without specifics, justifying a rating of 1.",
            "Mentioning public transport within reasonable distance is an implicit reference to accessibility.",
            "Noting basic services not too far away provides a vague description requiring interpretation."
        ],
        2: [
            "The precise mention of proximity to Central Park subway station and shopping centers offers explicit details on accessibility.",
            "Highlighting major highways and transportation hubs provides clear examples, warranting a rating of 2.",
            "Describing walking distance to downtown amenities gives unmistakable references to accessibility."
        ]
    },
    "New Investment-Remodeled or Updated": {
        1: [
            "Saying the house has had some updates over the years is an implicit mention without specifics, fitting rating 1.",
            "Referring to minor renovations recently made offers a broad description requiring interpretation.",
            "Noting that some parts have been refreshed is vague and lacks detail, justifying a rating of 1."
        ],
        2: [
            "Detailed mention of modern open-concept kitchen and marble countertops provides explicit examples, fitting rating 2.",
            "Highlighting new electrical wiring and plumbing systems gives clear details on renovations.",
            "Describing smart home technology upgrades offers unmistakable references to recent investments."
        ]
    },
    "Occupancy Status": {
        1: [
            "Saying the home is available soon is an implicit mention without specifics, suitable for rating 1.",
            "Expected readiness shortly is vague and lacks detail about occupancy status.",
            "Potential for quick move-in after purchase requires interpretation, aligning with rating 1."
        ],
        2: [
            "Stating current vacancy and immediate move-in availability provides explicit details on occupancy status.",
            "Ready for immediate occupancy with no arrangements needed gives clear examples, warranting rating 2.",
            "Move-in ready condition available after paperwork completion offers unmistakable references."
        ]
    },
    "Natural Environment": {
        1: [
            "Mentioning some parks nearby is a general statement without specifics, fitting rating 1.",
            "Green spaces within driving distance is vague and requires interpretation about natural environment.",
            "A few natural spots accessible from property lacks detail, justifying a rating of 1."
        ],
        2: [
            "Nestled near Riverwalk Park with trails offers explicit details about natural environment proximity.",
            "Surrounded by greenery with hiking trails gives clear examples, fitting rating 2.",
            "Close proximity to Lakeview Park provides unmistakable references to natural scenery."
        ]
    },
    "Size": {
        1: [
            "Stating the home is spacious enough for a small family is an implicit mention without specifics, suitable for rating 1.",
            "Adequate space for living arrangements is vague about size details.",
            "Sufficient room for basic needs requires interpretation, aligning with rating 1."
        ],
        2: [
            "Mentioning specific square footage and room count provides explicit details on size, fitting rating 2.",
            "Generous living space with expansive rooms offers clear examples about home size.",
            "Spacious layout with multiple living areas gives unmistakable references to size utilization."
        ]
    },
    "Signaling": {
        1: [
            "Charming home in a desirable area. This phrase uses positive language but lacks specific details or examples, making it implicit (rating 1).",
            "A must-see property! This statement signals interest but does not provide concrete information about why it should be seen (rating 1).",
            "Perfect starter home! This uses signaling language but does not specify what makes it perfect (rating 1)."
        ],
        2: [
            "This stunning property won't last long on the market! Unique architectural style and prime location. This sentence uses strong signaling by emphasizing uniqueness and location (rating 2).",
            "An exceptional find with unparalleled features! This highlights specific selling points using superlative language (rating 2).",
            "Luxury living at its finest in a prestigious neighborhood. This signals high value through luxury descriptors (rating 2)."
        ]
    },
    "Opinion": {
        1: [
            "Cozy and inviting atmosphere. This phrase conveys an opinion but lacks detailed justification or examples (rating 1).",
            "A great place to call home. This opinion suggests positivity but does not provide specific reasons (rating 1).",
            "Feels like home from the moment you step inside. This conveys sentiment without detailed backing (rating 1)."
        ],
        2: [
            "This property exudes warmth and character, offering an inviting atmosphere. This opinion is supported by descriptive language about warmth (rating 2).",
            "A harmonious blend of comfort and elegance throughout. This opinion uses specific qualities like comfort and elegance to justify its stance (rating 2).",
            "Every corner of this home speaks of quality craftsmanship. This statement supports its opinion with reference to craftsmanship (rating 2)."
        ]
    }
}

# Explanation of dictionary keys
key_explanations = {
    "text": "The specific mention or example from the description that relates to the context.",
    "rating": "The level of emphasis placed on the context, rated 1 or 2.",
    "reason": "The justification for the assigned rating, explaining why the text was rated as such."
}

In [ ]:
def make_example_reasons(context):
    example_reasons = []
    for rating, examples_list in examples[context].items():
        for idx, example in enumerate(examples_list):
            reason = justifications[context][rating]
            example_reasons.append(f"'{example}' - It is rated as {rating}. {reason[idx]}")
    return example_reasons

In [ ]:
make_example_reasons("Physical Housing Quality")

["'The house seems to be in decent shape, though it may require some touch-ups.' - It is rated as 1. The mention of the house being in decent shape but needing touch-ups suggests a general statement about the structure without specific details, fitting a rating of 1.",
 "'Overall, the property is in fair condition with potential for improvements.' - It is rated as 1. Describing the property as having potential for improvements implies a broad assessment of quality, which aligns with an implicit mention.",
 "'The structure appears solid, but some areas might need attention.' - It is rated as 1. Noting that some areas might need attention provides a vague reference to the house's condition, justifying a rating of 1.",
 "'The property boasts a brand-new roof and recently installed energy-efficient windows, ensuring top-notch physical housing quality.' - It is rated as 2. The specific mention of a brand-new roof and energy-efficient windows provides clear examples of high-quality features,

In [ ]:
def write_instruct(context):
    other_contexts = [c for c in contexts.keys() if c != context]
    prompt = " ".join([
        "<Data> You will be given information about a house: <Address> and <Description>. </Data>",
        f"<Task> Read the <Description> and determine whether it emphasizes the following aspect: {context} ({contexts[context]}).",
        f"The extracted text must be directly related to the {context} context. Otherwise exclude it completely from the output and do not assign any rating to it. </Task>",
        f"<Rating> Use this rating scale: {rating_scale}. </Rating>",
        f"<Example> Here are examples for {context} context: {'; '.join(make_example_reasons(context))}",
        f"Please generate your own reasons for the rating: Must contain 'Why this text is related to {context} context', and 'Why it is rated as 1 or 2'. </Example>",
        "<Output> Provide your answer strictly in list format on a single line without any additional comments or explanations:",
        "[{'text': str, 'rating': int, 'reason': str}, {second}, {third}, ...]",
        "If no relevant mention is found, answer 'No relevant texts' only. </Output>",
        f"<Explanation> Key explanations: {key_explanations} </Explanation>"
    ])
    return prompt

In [ ]:
write_instruct("Physical Housing Quality")

"<Data> You will be given information about a house: <Address> and <Description>. </Data> <Task> Read the <Description> and determine whether it emphasizes the following aspect: Physical Housing Quality (Describes the structural condition and quality of the house itself, including materials and build quality.). The extracted text must be directly related to the Physical Housing Quality context. Otherwise exclude it completely from the output and do not assign any rating to it. </Task> <Rating> Use this rating scale: {'1': 'Implicit mention or general statement without specific details.', '2': 'Explicit mention or detailed emphasis with specific examples.'}. </Rating> <Example> Here are examples for Physical Housing Quality context: 'The house seems to be in decent shape, though it may require some touch-ups.' - It is rated as 1. The mention of the house being in decent shape but needing touch-ups suggests a general statement about the structure without specific details, fitting a ratin

In [ ]:
def write_input(listing):
    prompt = " ".join([
        f"<Address> {listing['address']} </Address>",
        f"<Description> {listing['description']} </Description>",
    ])
    return prompt

In [ ]:
write_input(zillow_df.iloc[0])

"<Address> 10905 S Avenue H </Address> <Description> Tastefully and professionally updated 4BR / 3.5BA Brick Cap Cod property.  MAIN-LEVEL OFFERS= Functional Open Space Kitchen / Dining / Living Room Area Set-Up, with brand new white shaker style soft-close cabinets, clean white backsplash tile, modern white quartz countertop, top-of-the-line Samsung stainless steel appliances, hardwood floors throughout along with 1 bathroom and 2 spacious bedrooms with plenty of closet space. UPPER- LEVEL OFFERS= Dream master area with huge master bedroom (22x21), custom-built walk-in closet, and a true master bathroom (14x9) with elegant touches of whites and greys.  If the top 2 levels are not enough, the equally impressive LOWER-LEVEL OFFERS= An equally functional open area concept set-up with a huge family room / TV area (21x28), with easy access to a half bathroom, and what can also be considered/used as a second master bedroom with engineered flooring and it's own private bathroom and separate 

# Generate

In [ ]:
import re
from tqdm import tqdm
import time

def clean_parse(generated_text):
    pattern = r"\{'text':\s*'([^']*)',\s*'rating':\s*(\d+),\s*'reason':\s*'([^']*)'\}"
    matches = re.findall(pattern, generated_text)
    result_list = [{'text': match[0], 'rating': int(match[1]), 'reason': match[2]} for match in matches]
    return result_list

def validate_extraction(extracted_phrases, original_description):
    valid_phrases = []
    for phrase in extracted_phrases:
        if phrase['text'] in original_description:
            valid_phrases.append(phrase)
    return valid_phrases

def validate_result(result):
    if not isinstance(result, list): # List?
        return False
    if len(result) == 0: # Null?
        return False
    required_keys = {'text', 'rating', 'reason'}
    count_1, count_2 = 0, 0
    for item in result:
        if not isinstance(item, dict) or not required_keys.issubset(item.keys()): # Dictionary?
            return False
        rating = item.get('rating')
        if rating == 1:
            count_1 += 1
        elif rating == 2:
            count_2 += 1
        if count_1 > 10 or count_2 > 10: # Blow-up?
            return False
    return True

In [ ]:
import pandas as pd
from tqdm import tqdm
import time
import os

def llama_extract(df, output_path):
    df = df.copy()

    if os.path.exists(output_path):
        temp = pd.read_csv(output_path)
        last = str(temp['zpid'].iloc[-1])
        df = df[df.index >= df[df['zpid'].astype(int) == int(last)].index[0]]
    else:
        temp = pd.DataFrame()

    results = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):

        # Property Information
        property_type = "Single Family" if row['single'] == 0 else "Condo/Townhouse"
        property_info = [
            f"\n\n[Property Information]",
            f"- ID: {row['zpid']}",
            f"- TOM: {row['duration']} Days",
            f"- Type: {property_type}",
            f"- Address: {row['address']}, {row['city']}",
            f"- Description: {row['description']}\n"
        ]
        print("\n".join(property_info))

        # Llama Generation
        print("[Llama Generation]")
        row_result = {'zpid': row['zpid']}

        for context in contexts.keys():
            count_1, count_2 = 0, 0
            instruct = write_instruct(context)
            input_text = write_input(row)
            prompt = [{"role": "system", "content": instruct}, {"role": "user", "content": input_text}]

            # Generate and Check
            attempts = 0
            max_attempts = 3
            while attempts < max_attempts:
                generation = generator(
                    prompt, temperature=0.5, max_new_tokens=512, do_sample=True,
                    pad_token_id=generator.tokenizer.eos_token_id
                )
                result = generation[0]['generated_text'][-1]['content'] # Raw output
                if result == "No relevant texts":
                    result = [] # No relevant?
                    break
                result = clean_parse(result) # Right format?
                result = validate_extraction(result, row['description']) # Used term?
                if validate_result(result):
                    break
                attempts += 1
                print(f"(Validation failed for {context}. Regenerating... Attempt {attempts}/{max_attempts})")
            if not validate_result(result):
                print(f"(Failed to generate valid output for {context} after {max_attempts} attempts.)")
                result = []  # Default to empty list if all attempts fail

            count_1 = len([item for item in result if item.get('rating') == 1])
            count_2 = len([item for item in result if item.get('rating') == 2])

            row_result[context] = str(result)
            row_result[f"{context}_rating_1"] = count_1
            row_result[f"{context}_rating_2"] = count_2
            print(f"- {context} ({count_1}, {count_2}): {result}")

        results.append(row_result)
        combined_results = pd.concat([temp, pd.DataFrame(results)], ignore_index=True)
        combined_results.to_csv(output_path, index=False)

        # Pause
        print("")
        time.sleep(1)

    return pd.DataFrame(combined_results)

In [ ]:
from transformers import pipeline
from tqdm import tqdm
import time

instructs = {}
context_keys = list(contexts.keys())
for context in context_keys:
    instruct = write_instruct(context)
    instructs[context] = instruct

def batch_llama_extract(df, output_path, batch_size=16, max_attempts=3):

    # Existence
    df = df.copy()
    if os.path.exists(output_path):
        table = pd.read_csv(output_path)
        last = str(table['zpid'].iloc[-1])
        df = df[df.index >= df[df['zpid'].astype(int) == int(last)].index[0]]
    else:
        table = pd.DataFrame(columns=['zpid']+context_keys)

    # Loop
    num_batches = len(df) // batch_size + (len(df) % batch_size > 0)
    for batch_idx in tqdm(range(num_batches), desc='Llama Extract'):
        batch = df.iloc[batch_idx * batch_size:(batch_idx + 1) * batch_size]
        prompts_batch = {}
        valids_batch = {}
        outputs_batch = {}
        attempts = 0
        print(f"\nHouse Listings: {batch['zpid'].values}")

        # 1. Write Prompt
        for df_idx, listing in batch.iterrows():
            prompts = {}
            valids = {}
            outputs = {}
            input = write_input(listing)
            for context in context_keys:
                prompt = [{"role": "system", "content": instructs[context]}, {"role": "user", "content": input}]
                prompts[context] = prompt
                valids[context] = False
                outputs[context] = ''
            prompts_batch[df_idx] = prompts
            valids_batch[df_idx] = valids
            outputs_batch[df_idx] = outputs

        # 2. Get Answer
        while attempts < max_attempts:
            contexts_selected_batch = {}
            prompts_selected_batch = {}
            prompts_flatten_batch = []

            # 2-1. Contexts to Check
            for df_idx, listing in batch.iterrows():
                if valids_batch[df_idx] == {} or all(valids_batch[df_idx].values()):
                    continue # Already Done (Listing)
                contexts_selected = []
                prompts_selected = []
                for context in context_keys:
                    if not valids_batch[df_idx].get(context, True):
                        contexts_selected.append(context)
                        prompts_selected.append(prompts_batch[df_idx][context])
                contexts_selected_batch[df_idx] = contexts_selected
                prompts_selected_batch[df_idx] = prompts_selected
                prompts_flatten_batch.extend(prompts_selected)
            if len(prompts_flatten_batch) == 0:
                break # Already Done (Batch)
            else:
                print(f"{len(prompts_flatten_batch)}/{batch.shape[0]*len(context_keys)} Prompts Remained (Attempts {attempts+1}/{max_attempts})")

            # 2-2. Generate
            start_time = time.time()
            generations_batch = generator(
                prompts_flatten_batch, # Parallize with Listing(batch_size) * Context
                temperature=0.5,
                max_new_tokens=512,
                do_sample=True,
                pad_token_id=generator.tokenizer.eos_token_id
            )
            elapsed_time = time.time() - start_time
            minutes = int(elapsed_time // 60)
            seconds = int(elapsed_time % 60)
            print(f"Generated in {minutes:02d}:{seconds:02d}")

            # 2-3. Validation
            gen_idx = 0
            for df_idx, listing in batch.iterrows():
                contexts_selected = contexts_selected_batch[df_idx]
                generations = generations_batch[gen_idx:gen_idx+len(contexts_selected)]
                gen_idx += len(contexts_selected)
                for idx, generation in enumerate(generations):
                    valids_batch[df_idx][contexts_selected[idx]] = False
                    output = generation[0]['generated_text'][-1]['content'] # Raw Output
                    if output == "No relevant texts":
                        outputs_batch[df_idx][contexts_selected[idx]] = '' # No Relevant?
                        valids_batch[df_idx][contexts_selected[idx]] = True
                        continue # 1) Vaild: None
                    output = clean_parse(output)
                    output = validate_extraction(output, listing['description']) # Text in Description?
                    if validate_result(output): # Right Format?
                        outputs_batch[df_idx][contexts_selected[idx]] = str(output)
                        valids_batch[df_idx][contexts_selected[idx]] = True
                        continue # 2) Vaild: Correct
            attempts += 1

        # 3. Store
        remained = 0
        for df_idx, listing in batch.iterrows():
            outputs = outputs_batch[df_idx]
            table = pd.concat([table, pd.DataFrame(outputs, index=[0])], ignore_index=True)
            table.loc[df_idx, 'zpid'] = str(listing['zpid'])
            remained += sum(not valid for valid in valids_batch[df_idx].values())
        table.to_csv(output_path, index=False)
        if remained == 0:
            print(f"All {batch.shape[0]*len(context_keys)} Prompts Done")
        else:
            print(f"{remained}/{batch.shape[0]*len(context_keys)} Prompts Remained")
        time.sleep(1)

    return table

## Chicago

In [ ]:
# output_path = os.path.join(main_dir, 'dataset', '6. llama_extracted_ch.csv')
# llama_ch = llama_extract(zillow_df[zillow_df['city'] == 'CH'], output_path)

  0%|          | 0/3315 [00:00<?, ?it/s]



[Property Information]
- ID: 4171164
- TOM: 134 Days
- Type: Single Family
- Address: 10905 S Avenue H, CH
- Description: Tastefully and professionally updated 4BR / 3.5BA Brick Cap Cod property.  MAIN-LEVEL OFFERS= Functional Open Space Kitchen / Dining / Living Room Area Set-Up, with brand new white shaker style soft-close cabinets, clean white backsplash tile, modern white quartz countertop, top-of-the-line Samsung stainless steel appliances, hardwood floors throughout along with 1 bathroom and 2 spacious bedrooms with plenty of closet space. UPPER- LEVEL OFFERS= Dream master area with huge master bedroom (22x21), custom-built walk-in closet, and a true master bathroom (14x9) with elegant touches of whites and greys.  If the top 2 levels are not enough, the equally impressive LOWER-LEVEL OFFERS= An equally functional open area concept set-up with a huge family room / TV area (21x28), with easy access to a half bathroom, and what can also be considered/used as a second master bedro

  0%|          | 0/3315 [02:07<?, ?it/s]


KeyboardInterrupt: 

## New York

In [ ]:
# output_path = os.path.join(main_dir, 'dataset', '6. llama_extracted_ny.csv')
# llama_extract(zillow_df[zillow_df['city'] == 'NY'], output_path)

  0%|          | 0/226 [00:00<?, ?it/s]



[Property Information]
- ID: 32347743
- TOM: 243 Days
- Type: Single Family
- Address: 58 Elmwood Ave, NY
- Description: This 3 bedroom, semi attached home has many upgrades: 
Home was re-insulated with 5/8 sheet rock, 
1 yr old furnace with humidifier, 
Tankless water heater. Roof, siding, windows, sliding glass doors in kitchen 2 years old. Mitsubishi split 1st floor ac/heat in living and dining room. Newer kitchen, granite counters, stainless steel appliances. porcelain floor.  Wood burning stove with 5/8 fire proof sheet rock walls. Freshly painted, hardwood floors, crown molding, new plumbing and faucets under sinks. Attic fully floored with attic fan.  Call for your appt today.

[Llama Generation]
(Validation failed for Physical Housing Quality. Regenerating... Attempt 1/3)
- Physical Housing Quality (0, 8): [{'text': 'Home was re-insulated with 5/8 sheet rock', 'rating': 2, 'reason': 'Explicit mention of insulation, a key aspect of physical housing quality.'}, {'text': 'Roof, 

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


(Validation failed for Size. Regenerating... Attempt 2/3)
(Validation failed for Size. Regenerating... Attempt 3/3)
(Failed to generate valid output for Size after 3 attempts.)
- Size (0, 0): []
- Signaling (0, 6): [{'text': 'Tankless water heater', 'rating': 2, 'reason': 'This text is related to Signaling as it highlights a desirable feature of the property, implying energy efficiency and cost savings. It is rated as 2 because it uses specific details to emphasize the unique selling point of the tankless water heater.'}, {'text': 'Mitsubishi split 1st floor ac/heat in living and dining room', 'rating': 2, 'reason': 'This text is related to Signaling as it highlights a desirable feature of the property, implying comfort and convenience. It is rated as 2 because it uses specific details to emphasize the unique selling point of the Mitsubishi split AC/heat system.'}, {'text': 'Newer kitchen, granite counters, stainless steel appliances', 'rating': 2, 'reason': 'This text is related to Si

  0%|          | 1/226 [04:06<15:23:57, 246.39s/it]

...

 99%|█████████▉| 224/226 [12:24:00<07:10, 215.14s/it]



[Property Information]
- ID: 219693698
- TOM: 247 Days
- Type: Condo/Townhouse
- Address: 47 E 88th St #14A, NY
- Description: Triple mint, high floor, light-filled 2 bedrooms and a library in first class Carnegie Hill prewar building. One block from Central Park on Manhattan's Upper East Side, this 2-3 bedroom, 2.5 bath apartment is very recently renovated. It is in a well-managed and well-maintained coop which has street level retail income and low monthly maintenance fees.

The living room has west and south-facing windows and a wood-burning fireplace. It has beautiful original hand-pegged hardwood floors. There is a comfortable private library (or 3rd bedroom). All rooms have motorized window treatments.

The serene primary bedroom has a peek at Central Park and beautifully customized closets. The ensuite bath has a deep soaking tub, a large separate shower, a vanity with double sinks and heated floors. The second bedroom also has an ensuite bath with heated floors.

The large we

100%|█████████▉| 225/226 [12:28:36<03:53, 233.19s/it]



[Property Information]
- ID: 58579303
- TOM: 357 Days
- Type: Condo/Townhouse
- Address: 314 Loretto Street, NY
- Description: Looking for a great opportunity? Look no more! This property has tons of potential and is average in living space size for the neighborhood. It is located close to main roads with easy access to local amenities, such as shopping, banking, schools, and various eateries. Per our Appraisal report describe the subject property is a MFR (colonial style) built in 1999. It has approx. 2250q. ft. and has 4 bedrooms and 3.5 bathroom, with full/unfinished basement and 1-Att garage on 3750 sq lot.

[Llama Generation]
(Validation failed for Physical Housing Quality. Regenerating... Attempt 1/3)
(Validation failed for Physical Housing Quality. Regenerating... Attempt 2/3)
(Validation failed for Physical Housing Quality. Regenerating... Attempt 3/3)
(Failed to generate valid output for Physical Housing Quality after 3 attempts.)
- Physical Housing Quality (0, 0): []
(Valid

100%|██████████| 226/226 [12:32:45<00:00, 199.85s/it]


## Los Angeles

In [ ]:
# output_path = os.path.join(main_dir, 'dataset', '6. llama_extracted_la.csv')
# llama_la = llama_extract(zillow_df[zillow_df['city'] == 'LA'], output_path)

  0%|          | 0/218 [00:00<?, ?it/s]



[Property Information]
- ID: 20508540
- TOM: 105 Days
- Type: Single Family
- Address: 1348 Club View Dr, LA
- Description: Majestic design, sensational amenities, and verdant vistas of the Los Angeles Country Club golf course add up to refined luxury. With the signature vision of designer Aleck Dugally, the gated residence is exceptionally spacious, with double height ceilings, sweeping curves, fine woodwork, impressive fireplaces, and high-end fixtures and features for a luxurious lifestyle. From the main level and the upper floor, rooms open onto wide balustrade balconies that overlook the 9th hole tee and the surrounding acres of manicured landscape. The grand chandelier foyer, formal living room, and formal dining room suggest elegant entertaining, easily choreographed in the expansive chef's kitchen. The main floor also has a distinguished paneled library/office, and all three levels are served by an elevator. The bedrooms upstairs include a magnificent dual primary suite compl

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


(Validation failed for Natural Environment. Regenerating... Attempt 1/3)
(Validation failed for Natural Environment. Regenerating... Attempt 2/3)
(Validation failed for Natural Environment. Regenerating... Attempt 3/3)
(Failed to generate valid output for Natural Environment after 3 attempts.)
- Natural Environment (0, 0): []
- Size (0, 7): [{'text': 'exceptionally spacious', 'rating': 2, 'reason': 'Explicit mention of spaciousness with specific details about double height ceilings and sweeping curves.'}, {'text': 'wide balustrade balconies', 'rating': 2, 'reason': 'Explicit mention of size with specific examples of room layout and outdoor spaces.'}, {'text': 'distinguished paneled library/office', 'rating': 2, 'reason': 'Explicit mention of size with specific examples of room layout.'}, {'text': 'magnificent dual primary suite', 'rating': 2, 'reason': 'Explicit mention of size with specific examples of room layout and amenities.'}, {'text': 'two more bedroom suites', 'rating': 2, 'rea

  0%|          | 1/218 [04:39<16:52:14, 279.88s/it]

...

 99%|█████████▉| 216/218 [12:53:14<06:15, 187.93s/it]



[Property Information]
- ID: 302794497
- TOM: 78 Days
- Type: Single Family
- Address: 12678 Millennium, LA
- Description: Embrace luxury living and the best in Playa Vista with this exceptional 4-bedroom Collection home boasting two primary suites and an elevator for ultimate convenience. Ideally situated on Millennium Drive, revel in expansive views of the serene Common Grounds park, just steps away from the vibrant Runway Retail and Farmer's Market. This modern masterpiece, with its 10-foot ceilings and oversized windows, bathes every corner in natural light, accentuating the wide-plank engineered wood floors that exude opulence throughout, contrasted with plush carpeting in the bedrooms. The smooth finish walls add an air of elegance and sophistication to the ambiance. The main level welcomes you with an open-concept floor plan, seamlessly integrating the kitchen, dining, and living areas. Large glass sliding doors lead to the private patio and side yard, perfect for indoor-outdo

100%|█████████▉| 217/218 [12:59:27<04:03, 243.64s/it]



[Property Information]
- ID: 20463351
- TOM: 71 Days
- Type: Single Family
- Address: 2587 S Bundy Dr, LA
- Description: Welcome to 2587 S. Bundy, where charm meets endless potential in this sweet three-bedroom single-family home. You will love the classic oak hardwood floors, brick framed fireplace, and formal dining room that seamlessly connects to the kitchen and main living space. The property is set back beautifully offering a lovely front yard tucked behind greenery and a large backyard with a detached two car garage. With its convenient location, this property offers easy access to all that the Westside has to offer. Don't miss your chance to make this charming home your own!

[Llama Generation]
- Physical Housing Quality (5, 0): [{'text': 'classic oak hardwood floors', 'rating': 1, 'reason': 'This text is related to Physical Housing Quality as it mentions the presence of hardwood floors, which is a characteristic of a house.'}, {'text': 'brick framed fireplace', 'rating': 1, 

100%|██████████| 218/218 [13:01:55<00:00, 215.21s/it]


# Merge

In [3]:
llama_ch = pd.read_csv(os.path.join(main_dir, 'dataset', '6. llama_extracted_ch.csv'))
llama_ny = pd.read_csv(os.path.join(main_dir, 'dataset', '6. llama_extracted_ny.csv'))
llama_la = pd.read_csv(os.path.join(main_dir, 'dataset', '6. llama_extracted_la.csv'))

In [7]:
import numpy as np

city_to_df = {
    'CH': llama_ch,
    'NY': llama_ny,
    'LA': llama_la,
}

llama_dict = {}

for city in ['CH', 'NY', 'LA']:
    for single in [0, 1]:
        zpid_values = zillow_df[(zillow_df['city'] == city) & (zillow_df['single'] == single)]['zpid'].values
        city_to_df[city]['zpid'] = city_to_df[city]['zpid'].astype(str)
        table = city_to_df[city][city_to_df[city]['zpid'].isin(zpid_values)]

        cols = [col for col in table.columns if col.endswith('_rating_1') or col.endswith('_rating_2') or col == 'zpid']
        table = table[cols].reset_index(drop=True)

        key_freq = f'freq_{city}_{single}'
        llama_dict[key_freq] = table

        table_binary = table.drop(columns='zpid')
        binary_array = np.where(table_binary > 0, 1, 0)
        key_binary = f'binary_{city}_{single}'
        table_binary = pd.DataFrame(binary_array, columns=table_binary.columns)
        table_binary.insert(0, 'zpid', table['zpid'])
        llama_dict[key_binary] = table_binary

for key, df in llama_dict.items():
    df.columns = [col.replace('_rating_', ' ') for col in df.columns]

file_path = os.path.join(main_dir, 'dataset', '6. llama_extracted.h5')
for key, df in llama_dict.items():
    df.to_hdf(file_path, key=key, mode='a', format='table')

In [8]:
llama_dict['freq_CH_0']

,zpid,Physical Housing Quality 1,Physical Housing Quality 2,Neighborhood Characteristics 1,Neighborhood Characteristics 2,Accessibility 1,Accessibility 2,New Investment-Remodeled or Updated 1,New Investment-Remodeled or Updated 2,Occupancy Status 1,Occupancy Status 2,Natural Environment 1,Natural Environment 2,Size 1,Size 2,Signaling 1,Signaling 2,Opinion 1,Opinion 2
0,4171164,0,0,2,0,1,0,0,1,0,0,1,0,0,0,1,1,10,0
1,4016685,3,0,0,0,0,1,1,3,0,0,0,4,0,0,1,3,1,1
2,3638932,2,0,1,0,0,0,0,0,0,0,1,0,5,0,1,0,0,0
3,4116360,0,0,7,0,0,0,1,1,2,2,0,0,3,1,1,3,7,0
4,3946804,6,0,5,2,0,1,0,2,0,0,5,1,1,0,2,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4147854,1,0,1,0,3,0,5,0,3,0,0,0,1,0,0,0,1,0
1512,4020822,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,7,5
1513,159464747,4,0,5,0,1,0,0,1,0,0,1,0,4,2,1,3,1,0
1514,3619052,0,1,2,0,0,0,0,1,0,0,0,0,3,0,2,0,2,0


In [9]:
llama_dict['binary_CH_0']

,zpid,Physical Housing Quality 1,Physical Housing Quality 2,Neighborhood Characteristics 1,Neighborhood Characteristics 2,Accessibility 1,Accessibility 2,New Investment-Remodeled or Updated 1,New Investment-Remodeled or Updated 2,Occupancy Status 1,Occupancy Status 2,Natural Environment 1,Natural Environment 2,Size 1,Size 2,Signaling 1,Signaling 2,Opinion 1,Opinion 2
0,4171164,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,1,1,0
1,4016685,1,0,0,0,0,1,1,1,0,0,0,1,0,0,1,1,1,1
2,3638932,1,0,1,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0
3,4116360,0,0,1,0,0,0,1,1,1,1,0,0,1,1,1,1,1,0
4,3946804,1,0,1,1,0,1,0,1,0,0,1,1,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1511,4147854,1,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,1,0
1512,4020822,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,1,1
1513,159464747,1,0,1,0,1,0,0,1,0,0,1,0,1,1,1,1,1,0
1514,3619052,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,1,0


In [ ]:
import pandas as pd

# Assuming llama_dict and zillow_df are already defined as per your context

freq_descriptions = []

for city in ['CH', 'NY', 'LA']:
    for single in [0, 1]:
        key_freq = f'freq_{city}_{single}'
        freq_df = llama_dict[key_freq]

        # Sort zillow_df by duration and split into thirds
        temp = zillow_df[zillow_df['zpid'].isin(freq_df['zpid'])].copy()
        temp_sorted = temp.sort_values(by='duration')

        n = len(temp_sorted)
        # lower_third_zpids = temp_sorted.iloc[:n//3]['zpid']
        # middle_third_zpids = temp_sorted.iloc[n//3:2*n//3]['zpid']
        # upper_third_zpids = temp_sorted.iloc[2*n//3:]['zpid']

        lower_third_zpids = temp_sorted.iloc[:n//10]['zpid']
        middle_third_zpids = temp_sorted.iloc[n//10:9*n//10]['zpid']
        upper_third_zpids = temp_sorted.iloc[9*n//10:]['zpid']

        # Calculate mean for each third in freq_df
        for name, zpids in zip(['lower', 'middle', 'upper'], [lower_third_zpids, middle_third_zpids, upper_third_zpids]):
            group = freq_df[freq_df['zpid'].isin(zpids)]
            group_description = group.describe().loc[['mean']]
            group_description_t = group_description.T.assign(city=city, single=single, group=name)
            freq_descriptions.append(group_description_t)

# Combine all descriptions into a DataFrame
freq_descriptions_df = pd.concat(freq_descriptions)

# Reshape and process the DataFrame
reshaped_df = freq_descriptions_df.reset_index().melt(id_vars=['city', 'single', 'group', 'index'])
reshaped_df = reshaped_df.pivot_table(index='index', columns=['city', 'single', 'group', 'variable'], values='value')
reshaped_df.columns = [f'{city}_{single}_{group}_{stat}' for city, single, group, stat in reshaped_df.columns]
sorted_columns = sorted(reshaped_df.columns, key=lambda col: (['CH', 'NY', 'LA'].index(col.split('_')[0]), col))
reshaped_df = reshaped_df[sorted_columns]
reshaped_df = reshaped_df.round(3)

# Sort index if needed
reshaped_df_sorted = reshaped_df.sort_index()
reshaped_df_sorted

,CH_0_lower_mean,CH_0_middle_mean,CH_0_upper_mean,CH_1_lower_mean,CH_1_middle_mean,CH_1_upper_mean,NY_0_lower_mean,NY_0_middle_mean,NY_0_upper_mean,NY_1_lower_mean,NY_1_middle_mean,NY_1_upper_mean,LA_0_lower_mean,LA_0_middle_mean,LA_0_upper_mean,LA_1_lower_mean,LA_1_middle_mean,LA_1_upper_mean
index,,,,,,,,,,,,,,,,,,
Accessibility 1,1.263,1.405,1.784,1.511,1.564,1.634,1.515,1.406,1.495,1.285,1.370,1.342,1.308,1.412,1.365,1.382,1.700,1.565
Accessibility 2,0.283,0.277,0.157,0.433,0.452,0.459,0.297,0.384,0.437,0.545,0.470,0.500,0.277,0.277,0.255,0.293,0.339,0.306
Natural Environment 1,1.270,1.548,1.314,2.144,2.065,2.033,1.297,1.676,2.107,1.600,1.643,1.584,1.822,1.902,1.643,1.634,1.790,1.968
Natural Environment 2,0.099,0.145,0.092,0.250,0.263,0.273,0.178,0.186,0.194,0.275,0.227,0.134,0.308,0.385,0.353,0.325,0.229,0.129
Neighborhood Characteristics 1,1.914,2.125,2.039,2.022,2.256,2.109,1.812,2.059,2.427,1.760,1.975,2.129,2.004,1.895,2.020,2.301,2.029,2.419
Neighborhood Characteristics 2,0.237,0.259,0.176,0.256,0.220,0.262,0.139,0.251,0.233,0.230,0.280,0.208,0.289,0.289,0.235,0.236,0.315,0.210
New Investment-Remodeled or Updated 1,1.414,1.749,1.745,1.928,1.975,2.027,1.673,2.172,1.981,1.980,1.936,2.183,1.696,1.798,1.875,2.008,1.966,2.194
New Investment-Remodeled or Updated 2,1.507,1.730,1.281,1.228,1.412,1.503,0.653,1.250,1.155,0.880,1.094,0.752,1.241,1.257,1.161,0.935,1.446,0.863
Occupancy Status 1,0.987,0.930,1.092,0.900,0.977,1.120,0.861,0.941,0.922,0.680,0.910,1.089,0.834,0.644,0.596,0.626,0.841,1.081


In [11]:
## this

import pandas as pd

data = []

# Define percentiles for each city and single type combination
percentiles = {
    ('CH', 0, 'l'): 0.25,
    ('CH', 0, 'u'): 0.25,
    ('CH', 1, 'l'): 0.25,
    ('CH', 1, 'u'): 0.25,
    ('NY', 0, 'l'): 0.25,
    ('NY', 0, 'u'): 0.2,
    ('NY', 1, 'l'): 0.15,
    ('NY', 1, 'u'): 0.1,
    ('LA', 0, 'l'): 0.25,
    ('LA', 0, 'u'): 0.15,
    ('LA', 1, 'l'): 0.25,
    ('LA', 1, 'u'): 0.15
}

for city in ['CH', 'NY', 'LA']:
    for single in [0, 1]:
        key_binary = f'binary_{city}_{single}'
        binary_df = llama_dict[key_binary]

        # Sort zillow_df by duration and filter by zpid
        temp = zillow_df[zillow_df['zpid'].isin(binary_df['zpid'])].copy()
        temp_sorted = temp.sort_values(by='duration')

        n = len(temp_sorted)

        # Calculate indices based on percentiles
        lower_percentile = percentiles[(city, single, 'l')]
        upper_percentile = percentiles[(city, single, 'u')]

        lower_third_zpids = temp_sorted.iloc[:int(n * lower_percentile)]['zpid']
        upper_third_zpids = temp_sorted.iloc[int(n * (1 - upper_percentile)):]['zpid']
        middle_third_zpids = temp_sorted.iloc[int(n * lower_percentile):int(n * (1 - upper_percentile))]['zpid']

        # Calculate proportions for each third
        for name, zpids in zip(['lower', 'middle', 'upper'], [lower_third_zpids, middle_third_zpids, upper_third_zpids]):
            group = binary_df[binary_df['zpid'].isin(zpids)]
            proportions = group.iloc[:, 1:].sum() / len(group)
            data.append(pd.DataFrame(proportions).T.assign(city=city, single=single, group=name))

# Combine all proportions into a DataFrame
proportions_df = pd.concat(data, ignore_index=True)

# Reshape and process the DataFrame
proportions_df.set_index(['city', 'single', 'group'], inplace=True)
long_format_df = proportions_df.stack().reset_index()
long_format_df['city_single_group'] = long_format_df['city'].str.lower() + '_' + long_format_df['single'].astype(str) + '_' + long_format_df['group']
long_format_df = long_format_df.drop(columns=['city', 'single', 'group'])
long_format_df.columns = ['attribute', 'proportion', 'city_single_group']

# Pivot to get the desired format
final_long_format_df = long_format_df.pivot(index='attribute', columns='city_single_group', values='proportion')
sorted_columns = sorted(final_long_format_df.columns, key=lambda col: (['ch', 'ny', 'la'].index(col.split('_')[0]), col))
final_long_format_df = final_long_format_df[sorted_columns]
final_long_format_df = final_long_format_df.round(3)

final_long_format_df

city_single_group,ch_0_lower,ch_0_middle,ch_0_upper,ch_1_lower,ch_1_middle,ch_1_upper,ny_0_lower,ny_0_middle,ny_0_upper,ny_1_lower,ny_1_middle,ny_1_upper,la_0_lower,la_0_middle,la_0_upper,la_1_lower,la_1_middle,la_1_upper
attribute,,,,,,,,,,,,,,,,,,
Accessibility 1,0.595,0.570,0.592,0.588,0.609,0.626,0.600,0.552,0.556,0.543,0.558,0.525,0.558,0.580,0.615,0.625,0.679,0.591
Accessibility 2,0.214,0.219,0.176,0.322,0.334,0.327,0.251,0.302,0.332,0.393,0.359,0.351,0.192,0.199,0.183,0.256,0.229,0.237
Natural Environment 1,0.579,0.602,0.511,0.754,0.728,0.699,0.596,0.623,0.644,0.620,0.628,0.564,0.689,0.681,0.652,0.634,0.671,0.656
Natural Environment 2,0.098,0.099,0.066,0.182,0.173,0.160,0.149,0.114,0.141,0.177,0.166,0.099,0.211,0.246,0.212,0.155,0.150,0.113
Neighborhood Characteristics 1,0.775,0.764,0.811,0.805,0.832,0.835,0.765,0.779,0.815,0.767,0.767,0.807,0.765,0.746,0.796,0.809,0.798,0.849
Neighborhood Characteristics 2,0.153,0.179,0.145,0.169,0.154,0.158,0.176,0.159,0.176,0.160,0.189,0.144,0.197,0.197,0.175,0.223,0.205,0.183
New Investment-Remodeled or Updated 1,0.590,0.608,0.579,0.616,0.608,0.626,0.635,0.659,0.663,0.643,0.603,0.639,0.582,0.579,0.594,0.602,0.604,0.613
New Investment-Remodeled or Updated 2,0.638,0.650,0.647,0.596,0.619,0.600,0.478,0.511,0.434,0.453,0.510,0.436,0.563,0.559,0.508,0.524,0.595,0.457
Occupancy Status 1,0.479,0.450,0.453,0.408,0.402,0.468,0.408,0.443,0.473,0.370,0.426,0.460,0.361,0.301,0.327,0.288,0.348,0.419


In [ ]:
df = final_long_format_df.reset_index().copy()
cols = df.columns[1:]

for idx, row in df.iterrows():
    if idx%2 == 0:
        for col in cols:
            df.loc[idx, col] = (row[col] > 0.8)
    elif idx%2 ==1:
        for col in cols:
            df.loc[idx, col] = (row[col] > 0.2)

df

<ipython-input-32-0aa9889a55ba>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, col] = (row[col] > 0.8)
<ipython-input-32-0aa9889a55ba>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, col] = (row[col] > 0.8)
<ipython-input-32-0aa9889a55ba>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[idx, col] = (row[col] > 0.8)
<ipython-input-32-0aa9889a55ba>:7: FutureWarning: Setting an item of incompatible dtype is deprecated and 

city_single_group,attribute,ch_0_lower,ch_0_middle,ch_0_upper,ch_1_lower,ch_1_middle,ch_1_upper,ny_0_lower,ny_0_middle,ny_0_upper,ny_1_lower,ny_1_middle,ny_1_upper,la_0_lower,la_0_middle,la_0_upper,la_1_lower,la_1_middle,la_1_upper
0,Accessibility 1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,Accessibility 2,False,True,False,True,True,True,True,True,True,True,True,True,False,False,False,True,True,True
2,Natural Environment 1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,Natural Environment 2,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False
4,Neighborhood Characteristics 1,False,False,True,False,True,True,False,False,True,False,False,True,False,False,False,True,False,True
5,Neighborhood Characteristics 2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
6,New Investment-Remodeled or Updated 1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
7,New Investment-Remodeled or Updated 2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
8,Occupancy Status 1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
9,Occupancy Status 2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
